# RESTAURANT - Feature Engineering and Feature Selection

In [1]:
from scipy import stats
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import calendar
import os
import statsmodels.api as sm
import matplotlib.transforms as mtransforms
from scipy import stats
import warnings
from statsmodels.tsa.stattools import adfuller
from IPython.display import Image
import logging
from fbprophet import Prophet
import warnings
warnings.simplefilter('ignore')
import multiprocessing
from multiprocessing import Pool

warnings.simplefilter('ignore')

In [2]:
from sklearn import preprocessing

In [3]:
# global vars
# plt.rcParams.keys()
plt.rcParams['grid.linestyle'] ='--'
plt.rcParams['grid.alpha'] = 0.4
plt.rcParams['figure.figsize'] = [7,3]
plt.rcParams["font.weight"] = "light"
plt.rcParams["axes.titleweight"] = "bold"

sns.set_palette("PRGn", 10)

In [4]:
%matplotlib inline

In [5]:
from IPython.display import display_html

def display_side_by_side(*args):
    html_str=''
    for df in args:
        html_str+=df.to_html()
    display_html(html_str.replace('table','table style="display:inline"'),raw=True)

# IMPORT DATA

In [6]:
path_input = ('../input/')

In [7]:
!ls -1 {path_input}

air_reserve.csv.gz
air_store_info.csv.gz
air_visit_data.csv.gz
date_info.csv.gz
hpg_reserve.csv.gz
hpg_store_info.csv.gz
sample_submission.csv.gz
store_id_relation.csv.gz


In [8]:
air_reserve = pd.read_csv(os.path.join(path_input,'air_reserve.csv.gz'))
air_store_info = pd.read_csv(os.path.join(path_input,'air_store_info.csv.gz'))
air_visit_data = pd.read_csv(os.path.join(path_input,'air_visit_data.csv.gz'))
date_info = pd.read_csv(os.path.join(path_input,'date_info.csv.gz'))
hpg_reserve = pd.read_csv(os.path.join(path_input,'hpg_reserve.csv.gz'))
hpg_store_info = pd.read_csv(os.path.join(path_input,'hpg_store_info.csv.gz'))
sample_submission = pd.read_csv(os.path.join(path_input,'sample_submission.csv.gz'))
store_id_relation = pd.read_csv(os.path.join(path_input,'store_id_relation.csv.gz'))

# Feature engineering

#### Feature engineering steps:

create a seperate dataframe for feature enginnering (say train and test)

for each feature:
- create feature
- rename columns
- merge on train test dfs

# VISIT TABLE

In [9]:
train = air_visit_data.copy()
test = sample_submission.copy()

In [10]:
test['air_store_id'] = test['id'].apply(lambda x: '_'.join(x.split('_')[:2]))

test['visit_date'] = test['id'].apply(lambda x: x.split('_')[2])

In [11]:
train.air_store_id.nunique(),test.air_store_id.nunique()

(829, 821)

In [12]:
#drop id's not in test
train = train[train.air_store_id.isin(test.air_store_id.unique())]

## convert date format and create some date features

In [13]:
train['visit_date'] = pd.to_datetime(train['visit_date'])
train['year'] = train['visit_date'].dt.year
train['dow'] =  train['visit_date'].dt.dayofweek
train['dom'] = train['visit_date'].dt.day
train['month'] = train['visit_date'].dt.month
train['visit_date'] = train['visit_date'].dt.date

In [14]:
train.head()

,air_store_id,visit_date,visitors,year,dow,dom,month
0,air_ba937bf13d40fb24,2016-01-13,25,2016,2,13,1
1,air_ba937bf13d40fb24,2016-01-14,32,2016,3,14,1
2,air_ba937bf13d40fb24,2016-01-15,29,2016,4,15,1
3,air_ba937bf13d40fb24,2016-01-16,22,2016,5,16,1
4,air_ba937bf13d40fb24,2016-01-18,6,2016,0,18,1


In [15]:
test['visit_date'] = pd.to_datetime(test['visit_date'])
test['year'] = test['visit_date'].dt.year
test['dow'] = test['visit_date'].dt.dayofweek
test['dom'] = test['visit_date'].dt.day
test['month'] = test['visit_date'].dt.month
test['visit_date'] = test['visit_date'].dt.date

    Some restaurants at train dataset does not exit on test

In [16]:
len(set(train['air_store_id'].unique()) - set(test['air_store_id'].unique()))

0

## VISIT / DATE FEATURES

#### Day of week

In [17]:
aggregations = ['mean','median','min','max','std']

gr = train.groupby('dow')['visitors'].agg(aggregations)

gr.head()

gr.columns = ['dow_{}'.format(e) for e in gr.columns]

gr = gr.reset_index()

gr.head()

train = train.merge(gr,on='dow',how='left')

test = test.merge(gr,on='dow',how='left')

#### Day of month

In [18]:
aggregations = ['mean', 'median', 'min', 'max', 'std']

gr = train.groupby('dom')['visitors'].agg(aggregations)

gr.columns = ['dom_{}'.format(e) for e in gr.columns]

gr = gr.reset_index()

train = train.merge(gr, on='dom', how='left')

test = test.merge(gr, on='dom', how='left')


#### Month of year

In [19]:
aggregations = ['mean', 'median', 'min', 'max', 'std']

gr = train.groupby('month')['visitors'].agg(aggregations)

gr.columns = ['month_{}'.format(e) for e in gr.columns]

gr = gr.reset_index()

train = train.merge(gr, on='month', how='left')

test = test.merge(gr, on='month', how='left')

In [20]:
train.head()

,air_store_id,visit_date,visitors,year,dow,dom,month,dow_mean,dow_median,dow_min,...,dom_mean,dom_median,dom_min,dom_max,dom_std,month_mean,month_median,month_min,month_max,month_std
0,air_ba937bf13d40fb24,2016-01-13,25,2016,2,13,1,19.217106,16,1,...,19.254301,15,1,269,15.294143,19.968233,16,1,514,16.041859
1,air_ba937bf13d40fb24,2016-01-14,32,2016,3,14,1,18.898359,15,1,...,20.194978,16,1,372,15.855515,19.968233,16,1,514,16.041859
2,air_ba937bf13d40fb24,2016-01-15,29,2016,4,15,1,23.087137,19,1,...,21.241133,18,1,150,16.096449,19.968233,16,1,514,16.041859
3,air_ba937bf13d40fb24,2016-01-16,22,2016,5,16,1,26.310500,22,1,...,21.392962,17,1,207,16.755058,19.968233,16,1,514,16.041859
4,air_ba937bf13d40fb24,2016-01-18,6,2016,0,18,1,17.170329,14,1,...,21.869031,18,1,777,18.646356,19.968233,16,1,514,16.041859


## STORE FEATURES

#### Visitors by stores and days of weeks

In [21]:
stores_features = []

gr = train.groupby(['air_store_id','dow'])['visitors'].agg(['min','mean','median','max','count','std'])

In [22]:
gr.head(20)

min       mean  median  max  count        std
air_store_id         dow                                               
air_00a91d42b08b08d9 0      1  22.457143    19.0   47     35   9.425693
                     1      1  24.350000    24.5   43     40   9.093672
                     2     15  28.125000    28.0   52     40   8.881521
                     3     15  29.868421    30.0   47     38   7.143968
                     4     17  36.500000    35.5   57     40   9.021342
                     5      3  14.973684    11.0   99     38  16.793639
                     6      2   2.000000     2.0    2      1        NaN
air_0164b9927d20bcc3 0      2   7.500000     6.0   19     20   5.226451
                     1      1   9.560000     8.0   24     25   6.076457
                     2      2   9.678571     8.0   27     28   6.271946
                     3      1   9.846154    10.5   23     26   5.682903
                     4      3  11.464286    10.5   26     28   6.914730
                     5      1   6.409091     4.5   25     22   6.925860
air_0241aa3964b7f861 0      2   8.920635     8.0   23     63   4.790116
                     1      1   8.621212     8.0   34     66   5.524115
                     2      2   9.852941     9.0   24     68   4.902831
                     3      1   5.555556     3.0   18      9   6.227181
                     4      1   7.786885     6.0   48     61   6.762925
                     5      2  13.892308    13.0   37     65   7.235164
                     6      1  10.781250    10.0   26     64   5.826782

In [23]:
   
gr = gr.unstack(level=-1).fillna(0)

In [24]:
gr.head()

min                                        mean  \
dow                     0    1     2     3     4    5    6          0   
air_store_id                                                            
air_00a91d42b08b08d9  1.0  1.0  15.0  15.0  17.0  3.0  2.0  22.457143   
air_0164b9927d20bcc3  2.0  1.0   2.0   1.0   3.0  1.0  0.0   7.500000   
air_0241aa3964b7f861  2.0  1.0   2.0   1.0   1.0  2.0  1.0   8.920635   
air_0328696196e46f18  2.0  2.0   1.0   1.0   1.0  2.0  1.0   6.416667   
air_034a3d5b40d5b1b1  1.0  1.0   1.0   1.0   2.0  1.0  1.0  11.864865   

                                              ...     count              \
dow                           1          2    ...         4     5     6   
air_store_id                                  ...                         
air_00a91d42b08b08d9  24.350000  28.125000    ...      40.0  38.0   1.0   
air_0164b9927d20bcc3   9.560000   9.678571    ...      28.0  22.0   0.0   
air_0241aa3964b7f861   8.621212   9.852941    ...      61.0  65.0  64.0   
air_0328696196e46f18   7.285714   7.058824    ...      22.0  17.0  17.0   
air_034a3d5b40d5b1b1  10.888889  15.105263    ...      41.0  40.0  38.0   

                            std                                           \
dow                           0         1          2         3         4   
air_store_id                                                               
air_00a91d42b08b08d9   9.425693  9.093672   8.881521  7.143968  9.021342   
air_0164b9927d20bcc3   5.226451  6.076457   6.271946  5.682903  6.914730   
air_0241aa3964b7f861   4.790116  5.524115   4.902831  6.227181  6.762925   
air_0328696196e46f18   7.115391  4.339950   5.973594  7.139369  4.244681   
air_034a3d5b40d5b1b1  11.576226  8.594724  19.667217  8.480588  8.225629   

                                            
dow                           5          6  
air_store_id                                
air_00a91d42b08b08d9  16.793639   0.000000  
air_0164b9927d20bcc3   6.925860   0.000000  
air_0241aa3964b7f861   7.235164   5.826782  
air_0328696196e46f18   6.239697  10.607295  
air_034a3d5b40d5b1b1  12.134899  15.973841  

[5 rows x 42 columns]

In [25]:
gr.columns = ['STORE_VISITORS_BY_DATE_{}_{}'.format(e[0],e[1]) for e in gr.columns]

stores_features.extend(gr.columns)

gr = gr.reset_index()

gr.head()

,air_store_id,STORE_VISITORS_BY_DATE_min_0,STORE_VISITORS_BY_DATE_min_1,STORE_VISITORS_BY_DATE_min_2,STORE_VISITORS_BY_DATE_min_3,STORE_VISITORS_BY_DATE_min_4,STORE_VISITORS_BY_DATE_min_5,STORE_VISITORS_BY_DATE_min_6,STORE_VISITORS_BY_DATE_mean_0,STORE_VISITORS_BY_DATE_mean_1,...,STORE_VISITORS_BY_DATE_count_4,STORE_VISITORS_BY_DATE_count_5,STORE_VISITORS_BY_DATE_count_6,STORE_VISITORS_BY_DATE_std_0,STORE_VISITORS_BY_DATE_std_1,STORE_VISITORS_BY_DATE_std_2,STORE_VISITORS_BY_DATE_std_3,STORE_VISITORS_BY_DATE_std_4,STORE_VISITORS_BY_DATE_std_5,STORE_VISITORS_BY_DATE_std_6
0,air_00a91d42b08b08d9,1.0,1.0,15.0,15.0,17.0,3.0,2.0,22.457143,24.350000,...,40.0,38.0,1.0,9.425693,9.093672,8.881521,7.143968,9.021342,16.793639,0.000000
1,air_0164b9927d20bcc3,2.0,1.0,2.0,1.0,3.0,1.0,0.0,7.500000,9.560000,...,28.0,22.0,0.0,5.226451,6.076457,6.271946,5.682903,6.914730,6.925860,0.000000
2,air_0241aa3964b7f861,2.0,1.0,2.0,1.0,1.0,2.0,1.0,8.920635,8.621212,...,61.0,65.0,64.0,4.790116,5.524115,4.902831,6.227181,6.762925,7.235164,5.826782
3,air_0328696196e46f18,2.0,2.0,1.0,1.0,1.0,2.0,1.0,6.416667,7.285714,...,22.0,17.0,17.0,7.115391,4.339950,5.973594,7.139369,4.244681,6.239697,10.607295
4,air_034a3d5b40d5b1b1,1.0,1.0,1.0,1.0,2.0,1.0,1.0,11.864865,10.888889,...,41.0,40.0,38.0,11.576226,8.594724,19.667217,8.480588,8.225629,12.134899,15.973841


In [26]:
train = pd.merge(train, gr, how='left', on=['air_store_id']) 
test = pd.merge(test, gr, how='left', on=['air_store_id']) 

# STORE TABLE

## Label encode genres and areas

In [27]:
air_store_info.head()

,air_store_id,air_genre_name,air_area_name,latitude,longitude
0,air_0f0cdeee6c9bf3d7,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852
1,air_7cc17a324ae5c7dc,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852
2,air_fee8dcf4d619598e,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852
3,air_a17f0778617c76e2,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852
4,air_83db5aff8f50478e,Italian/French,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599


## Genre

In [28]:
air_store_info.air_genre_name.unique()

array(['Italian/French', 'Dining bar', 'Yakiniku/Korean food',
       'Cafe/Sweets', 'Izakaya', 'Okonomiyaki/Monja/Teppanyaki',
       'Bar/Cocktail', 'Japanese food', 'Creative cuisine', 'Other',
       'Western food', 'International cuisine', 'Asian', 'Karaoke/Party'],
      dtype=object)

In [29]:
#init label encoder
lbl = preprocessing.LabelEncoder()
# label encode genre
air_store_info['air_genre_name'] = lbl.fit_transform(air_store_info['air_genre_name'])

In [30]:
air_store_info.head()

,air_store_id,air_genre_name,air_area_name,latitude,longitude
0,air_0f0cdeee6c9bf3d7,6,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852
1,air_7cc17a324ae5c7dc,6,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852
2,air_fee8dcf4d619598e,6,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852
3,air_a17f0778617c76e2,6,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852
4,air_83db5aff8f50478e,6,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599


In [31]:
train = pd.merge(train, air_store_info[['air_store_id', 'air_genre_name']], how='left', on=['air_store_id']) 
test = pd.merge(test, air_store_info[['air_store_id', 'air_genre_name']], how='left', on=['air_store_id'])

#### Aggregations with genre

In [32]:
aggregations = ['mean', 'median', 'min', 'max', 'std']

gr = train.groupby('air_genre_name')['visitors'].agg(aggregations)

gr.columns = ['genre_{}'.format(e) for e in gr.columns]

gr = gr.reset_index()

In [33]:
train = train.merge(gr, on='air_genre_name',how='left')
test = test.merge(gr, on='air_genre_name',how='left')

## AREA

In [34]:
# for e in np.sort(air_store_info['air_area_name'].unique()):
#     print(e)

In [35]:
# http://www.wikizeroo.net/index.php?q=aHR0cHM6Ly9lbi53aWtpcGVkaWEub3JnL3dpa2kvQWRtaW5pc3RyYXRpdmVfZGl2aXNpb25zX29mX0phcGFu


In [36]:
air_store_info['air_area_name'] = air_store_info['air_area_name'].map(lambda x: str(str(x).replace('-',' ')))

In [37]:
air_store_info['air_area_name_1'] = air_store_info['air_area_name'].apply(lambda x: x.split()[0])

air_store_info['air_area_name_2'] = air_store_info['air_area_name'].apply(lambda x: x.split()[1])

air_store_info['air_area_name_3'] = air_store_info['air_area_name'].apply(lambda x: x.split()[2])

air_store_info['air_area_name_4'] = air_store_info['air_area_name'].apply(lambda x: x.split()[3])

air_store_info['air_area_name_5'] = air_store_info['air_area_name'].apply(lambda x: '_'.join(x.split()[3:]))

In [38]:
air_store_info.head()

,air_store_id,air_genre_name,air_area_name,latitude,longitude,air_area_name_1,air_area_name_2,air_area_name_3,air_area_name_4,air_area_name_5
0,air_0f0cdeee6c9bf3d7,6,Hyōgo ken Kōbe shi Kumoidōri,34.695124,135.197852,Hyōgo,ken,Kōbe,shi,shi_Kumoidōri
1,air_7cc17a324ae5c7dc,6,Hyōgo ken Kōbe shi Kumoidōri,34.695124,135.197852,Hyōgo,ken,Kōbe,shi,shi_Kumoidōri
2,air_fee8dcf4d619598e,6,Hyōgo ken Kōbe shi Kumoidōri,34.695124,135.197852,Hyōgo,ken,Kōbe,shi,shi_Kumoidōri
3,air_a17f0778617c76e2,6,Hyōgo ken Kōbe shi Kumoidōri,34.695124,135.197852,Hyōgo,ken,Kōbe,shi,shi_Kumoidōri
4,air_83db5aff8f50478e,6,Tōkyō to Minato ku Shibakōen,35.658068,139.751599,Tōkyō,to,Minato,ku,ku_Shibakōen


In [39]:
for e in [cols for cols in air_store_info.columns if 'air_area_name' in cols]:
        lbl = preprocessing.LabelEncoder()
        air_store_info[e] = lbl.fit_transform(air_store_info[e])

In [40]:
cols_area = [cols for cols in air_store_info.columns if 'air_area_name' in cols]

In [41]:
train = pd.merge(train, air_store_info[['air_store_id']+cols_area], how='left', on=['air_store_id']) 
test = pd.merge(test, air_store_info[['air_store_id']+cols_area], how='left', on=['air_store_id'])

# COORDINATE FEATURES

In [42]:
air_store_info = pd.read_csv(os.path.join(path_input,'air_store_info.csv.gz'))

In [43]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=10, random_state=0).fit(air_store_info[['longitude','latitude']])
air_store_info['loc_cluster'] = kmeans.predict(air_store_info[['longitude','latitude']])

In [83]:
for _,e in air_store_info.groupby('loc_cluster'):
    display(e.head(2))

,air_store_id,air_genre_name,air_area_name,latitude,longitude,loc_cluster,center_x,center_y,loc_distance_to_centroid
82,air_42c9aa6d617c5057,Italian/French,Hyōgo-ken Kakogawa-shi Kakogawachō Kitazaike,34.75695,134.841177,0,134.767366,34.784518,0.078792
83,air_645cb18b33f938cf,Italian/French,Hyōgo-ken Kakogawa-shi Kakogawachō Kitazaike,34.75695,134.841177,0,134.767366,34.784518,0.078792


,air_store_id,air_genre_name,air_area_name,latitude,longitude,loc_cluster,center_x,center_y,loc_distance_to_centroid
4,air_83db5aff8f50478e,Italian/French,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,1,139.712946,35.675963,0.042594
5,air_99c3eae84130c1cb,Italian/French,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,1,139.712946,35.675963,0.042594


,air_store_id,air_genre_name,air_area_name,latitude,longitude,loc_cluster,center_x,center_y,loc_distance_to_centroid
68,air_2c6fef1ce0e13a5a,Italian/French,Hokkaidō Sapporo-shi Minami 3 Jōnishi,43.05546,141.340956,2,141.345547,43.059303,0.005987
69,air_ca957d3a1529fbd3,Italian/French,Hokkaidō Sapporo-shi Minami 3 Jōnishi,43.05546,141.340956,2,141.345547,43.059303,0.005987


,air_store_id,air_genre_name,air_area_name,latitude,longitude,loc_cluster,center_x,center_y,loc_distance_to_centroid
17,air_4579cb0669fd411b,Italian/French,Fukuoka-ken Fukuoka-shi Daimyō,33.589216,130.392813,3,130.422985,33.586867,0.030263
18,air_63a750d8b4b6a976,Italian/French,Fukuoka-ken Fukuoka-shi Daimyō,33.589216,130.392813,3,130.422985,33.586867,0.030263


,air_store_id,air_genre_name,air_area_name,latitude,longitude,loc_cluster,center_x,center_y,loc_distance_to_centroid
176,air_91beafbba9382b0a,Dining bar,Miyagi-ken Sendai-shi Kamisugi,38.269076,140.870403,4,140.870403,38.269076,0.0
177,air_9efaa7ded03c5a71,Dining bar,Miyagi-ken Sendai-shi Kamisugi,38.269076,140.870403,4,140.870403,38.269076,0.0


,air_store_id,air_genre_name,air_area_name,latitude,longitude,loc_cluster,center_x,center_y,loc_distance_to_centroid
43,air_8e492076a1179383,Italian/French,Hiroshima-ken Hiroshima-shi Kokutaijimachi,34.386245,132.455018,5,132.653993,34.408214,0.200185
44,air_0867f7bebad6a649,Italian/French,Hiroshima-ken Hiroshima-shi Kokutaijimachi,34.386245,132.455018,5,132.653993,34.408214,0.200185


,air_store_id,air_genre_name,air_area_name,latitude,longitude,loc_cluster,center_x,center_y,loc_distance_to_centroid
66,air_6b15edd1b4fbb96a,Italian/French,Shizuoka-ken Hamamatsu-shi Motoshirochō,34.710895,137.72594,6,137.961885,34.797679,0.251399
67,air_6d65542aa43b598b,Italian/French,Shizuoka-ken Hamamatsu-shi Motoshirochō,34.710895,137.72594,6,137.961885,34.797679,0.251399


,air_store_id,air_genre_name,air_area_name,latitude,longitude,loc_cluster,center_x,center_y,loc_distance_to_centroid
412,air_17bed6dbf7c8b0fc,Cafe/Sweets,Niigata-ken Kashiwazaki-shi Chūōchō,37.3719,138.558984,7,138.927902,37.762398,0.537205
413,air_a239a44805932bab,Cafe/Sweets,Niigata-ken Kashiwazaki-shi Chūōchō,37.3719,138.558984,7,138.927902,37.762398,0.537205


,air_store_id,air_genre_name,air_area_name,latitude,longitude,loc_cluster,center_x,center_y,loc_distance_to_centroid
0,air_0f0cdeee6c9bf3d7,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,8,135.425983,34.699964,0.228182
1,air_7cc17a324ae5c7dc,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852,8,135.425983,34.699964,0.228182


,air_store_id,air_genre_name,air_area_name,latitude,longitude,loc_cluster,center_x,center_y,loc_distance_to_centroid
123,air_7ef9a5ea5c8fe39f,Dining bar,Hokkaidō Asahikawa-shi 6 Jōdōri,43.770635,142.364819,9,142.687374,43.708695,0.328449
124,air_8b4a46dc521bfcfe,Dining bar,Hokkaidō Asahikawa-shi 6 Jōdōri,43.770635,142.364819,9,142.687374,43.708695,0.328449


In [45]:
cluster_centers = pd.DataFrame(kmeans.cluster_centers_)
cluster_centers = cluster_centers.reset_index()
cluster_centers.columns = ['loc_cluster','center_x','center_y'] 

In [46]:
air_store_info = air_store_info.merge(cluster_centers,on='loc_cluster',how='left')

In [47]:
air_store_info['loc_distance_to_centroid'] = np.linalg.norm(air_store_info[['longitude','latitude']].values-
               air_store_info[['center_x','center_y']].values,axis=1)


In [48]:
loc_cols = [cols for cols in air_store_info if 'loc' in cols]

In [49]:
train = pd.merge(train, air_store_info[['air_store_id']+loc_cols], how='left', on=['air_store_id']) 
test = pd.merge(test, air_store_info[['air_store_id']+loc_cols], how='left', on=['air_store_id'])

# RESERVATION FEAUTURES

In [50]:
air_reserve.head()

,air_store_id,visit_datetime,reserve_datetime,reserve_visitors
0,air_877f79706adbfb06,2016-01-01 19:00:00,2016-01-01 16:00:00,1
1,air_db4b38ebe7a7ceff,2016-01-01 19:00:00,2016-01-01 19:00:00,3
2,air_db4b38ebe7a7ceff,2016-01-01 19:00:00,2016-01-01 19:00:00,6
3,air_877f79706adbfb06,2016-01-01 20:00:00,2016-01-01 16:00:00,2
4,air_db80363d35f10926,2016-01-01 20:00:00,2016-01-01 01:00:00,5


In [51]:
air_reserve['visit_datetime'] = pd.to_datetime(air_reserve['visit_datetime'])
air_reserve['reserve_datetime'] = pd.to_datetime(air_reserve['reserve_datetime'])
air_reserve['visit_datetime'] = air_reserve['visit_datetime'].dt.date
air_reserve['reserve_datetime'] = air_reserve['reserve_datetime'].dt.date
air_reserve['reserve_datetime_diff'] = air_reserve.apply(lambda x: (x['visit_datetime'] - x['reserve_datetime']).days, axis=1)

In [52]:
# initialize a list to store column names 
air_reserve_features = []
# aggregation list to aggregate
aggregations =  ['mean', 'min', 'max', 'sum', 'std', 'median']
# groupby and aggregataions
gr = air_reserve.groupby(['air_store_id','visit_datetime'])\
[['reserve_datetime_diff', 'reserve_visitors']].agg(aggregations)
# change column names
gr.columns = ['air_reserve_{}_{}'.format(e[0],e[1]) for e in gr.columns]
# add column nmes 
air_reserve_features.extend(gr.columns)
# prepare for merge
gr = gr.reset_index()
gr = gr.rename(columns={'visit_datetime':'visit_date'})

gr.head(2)

,air_store_id,visit_date,air_reserve_reserve_datetime_diff_mean,air_reserve_reserve_datetime_diff_min,air_reserve_reserve_datetime_diff_max,air_reserve_reserve_datetime_diff_sum,air_reserve_reserve_datetime_diff_std,air_reserve_reserve_datetime_diff_median,air_reserve_reserve_visitors_mean,air_reserve_reserve_visitors_min,air_reserve_reserve_visitors_max,air_reserve_reserve_visitors_sum,air_reserve_reserve_visitors_std,air_reserve_reserve_visitors_median
0,air_00a91d42b08b08d9,2016-10-31,0.0,0,0,0,NaN,0.0,2.0,2,2,2,NaN,2.0
1,air_00a91d42b08b08d9,2016-12-05,4.0,4,4,4,NaN,4.0,9.0,9,9,9,NaN,9.0


In [53]:
train = train.merge(gr, how='left', on=['air_store_id','visit_date'])
test = test.merge(gr, how='left', on=['air_store_id','visit_date'])

In [54]:
# sanity check
train[(train.air_store_id=='air_fee8dcf4d619598e')&(train.visit_date == pd.to_datetime('2017-01-09').date())]

,air_store_id,visit_date,visitors,year,dow,dom,month,dow_mean,dow_median,dow_min,...,air_reserve_reserve_datetime_diff_max,air_reserve_reserve_datetime_diff_sum,air_reserve_reserve_datetime_diff_std,air_reserve_reserve_datetime_diff_median,air_reserve_reserve_visitors_mean,air_reserve_reserve_visitors_min,air_reserve_reserve_visitors_max,air_reserve_reserve_visitors_sum,air_reserve_reserve_visitors_std,air_reserve_reserve_visitors_median
35703,air_fee8dcf4d619598e,2017-01-09,17,2017,0,9,1,17.170329,14,1,...,13.0,16.0,7.071068,8.0,2.5,2.0,3.0,5.0,0.707107,2.5


###    do the same for hpg tables

In [55]:
hpg_reserve.shape

(2000320, 4)

In [56]:
hpg_reserve.head()

,hpg_store_id,visit_datetime,reserve_datetime,reserve_visitors
0,hpg_c63f6f42e088e50f,2016-01-01 11:00:00,2016-01-01 09:00:00,1
1,hpg_dac72789163a3f47,2016-01-01 13:00:00,2016-01-01 06:00:00,3
2,hpg_c8e24dcf51ca1eb5,2016-01-01 16:00:00,2016-01-01 14:00:00,2
3,hpg_24bb207e5fd49d4a,2016-01-01 17:00:00,2016-01-01 11:00:00,5
4,hpg_25291c542ebb3bc2,2016-01-01 17:00:00,2016-01-01 03:00:00,13


In [57]:
# first, merge hpg_reserve and store_id table to put air_reserve_id info
store_id_relation.head(2)

,air_store_id,hpg_store_id
0,air_63b13c56b7201bd9,hpg_4bc649e72e2a239a
1,air_a24bf50c3e90d583,hpg_c34b496d0305a809


In [58]:
# we are losing lots of information since we keep less than 0.02 rows of the hpg_reserves 
# table!

# how='inner' keep restaurants that only hve both air and hpg ids
hpg_air_reserve = pd.merge(hpg_reserve,store_id_relation,how='inner', on=['hpg_store_id'])

In [59]:
hpg_air_reserve.shape

(28183, 5)

In [60]:
hpg_air_reserve.head(2)

,hpg_store_id,visit_datetime,reserve_datetime,reserve_visitors,air_store_id
0,hpg_878cc70b1abc76f7,2016-01-01 19:00:00,2016-01-01 15:00:00,4,air_db80363d35f10926
1,hpg_878cc70b1abc76f7,2016-01-02 19:00:00,2016-01-02 14:00:00,2,air_db80363d35f10926


    but we are losing lots of information since we keep less than 0.02 rows of the hpg_reserves table!

In [61]:
print(hpg_reserve.shape, hpg_air_reserve.shape, hpg_air_reserve.shape[0]/hpg_reserve.shape[0])

(2000320, 4) (28183, 5) 0.014089245720684691


In [62]:
hpg_air_reserve['visit_datetime'] = pd.to_datetime(hpg_air_reserve['visit_datetime'])
hpg_air_reserve['reserve_datetime'] = pd.to_datetime(hpg_air_reserve['reserve_datetime'])
hpg_air_reserve['reserve_datetime'] = hpg_air_reserve['reserve_datetime'].dt.date
hpg_air_reserve['visit_datetime'] = hpg_air_reserve['visit_datetime'].dt.date
hpg_air_reserve['reserve_datetime_diff'] = hpg_air_reserve.apply(lambda x: (
    x['visit_datetime'] - x['reserve_datetime']).days, axis=1)

In [63]:
# initialize a list to store column names
hpg_air_reserve_features = []
# aggregation list to aggregate
aggregations = ['mean', 'min', 'max', 'sum', 'std', 'median']
# groupby and aggregataions
gr = hpg_air_reserve.groupby(['air_store_id', 'visit_datetime'])[
    ['reserve_datetime_diff', 'reserve_visitors']].agg(aggregations)
# change column names
gr.columns = ['hpg_air_reserve_{}_{}'.format(e[0], e[1]) for e in gr.columns]
# add column nmes
hpg_air_reserve_features.extend(gr.columns)
# prepare for merge
gr = gr.reset_index()
gr = gr.rename(columns={'visit_datetime': 'visit_date'})

gr.head(2)

,air_store_id,visit_date,hpg_air_reserve_reserve_datetime_diff_mean,hpg_air_reserve_reserve_datetime_diff_min,hpg_air_reserve_reserve_datetime_diff_max,hpg_air_reserve_reserve_datetime_diff_sum,hpg_air_reserve_reserve_datetime_diff_std,hpg_air_reserve_reserve_datetime_diff_median,hpg_air_reserve_reserve_visitors_mean,hpg_air_reserve_reserve_visitors_min,hpg_air_reserve_reserve_visitors_max,hpg_air_reserve_reserve_visitors_sum,hpg_air_reserve_reserve_visitors_std,hpg_air_reserve_reserve_visitors_median
0,air_00a91d42b08b08d9,2016-01-14,3.0,3,3,3,NaN,3.0,2.0,2,2,2,NaN,2.0
1,air_00a91d42b08b08d9,2016-01-15,6.0,6,6,6,NaN,6.0,4.0,4,4,4,NaN,4.0


In [64]:
train = train.merge(gr, how='left', on=['air_store_id','visit_date'])
test = test.merge(gr, how='left', on=['air_store_id','visit_date'])

In [65]:
# train.head()

#### Merge reserve and visit tables

In [66]:
air_reserve.head()

,air_store_id,visit_datetime,reserve_datetime,reserve_visitors,reserve_datetime_diff
0,air_877f79706adbfb06,2016-01-01,2016-01-01,1,0
1,air_db4b38ebe7a7ceff,2016-01-01,2016-01-01,3,0
2,air_db4b38ebe7a7ceff,2016-01-01,2016-01-01,6,0
3,air_877f79706adbfb06,2016-01-01,2016-01-01,2,0
4,air_db80363d35f10926,2016-01-01,2016-01-01,5,0


In [67]:
air_reserve = air_reserve.rename(columns={'visit_datetime':'visit_date'})

In [68]:
train = pd.merge(train, air_reserve, how='left', on=['air_store_id','visit_date']) 
test = pd.merge(test, air_reserve, how='left', on=['air_store_id','visit_date'])

In [69]:
# train['id'] = train.apply(lambda r: '_'.join([str(r['air_store_id']), str(r['visit_date'])]), axis=1)

#### Holiday

In [70]:
date_info.head()

,calendar_date,day_of_week,holiday_flg
0,2016-01-01,Friday,1
1,2016-01-02,Saturday,1
2,2016-01-03,Sunday,1
3,2016-01-04,Monday,0
4,2016-01-05,Tuesday,0


In [71]:
date_info = date_info.rename(columns={'calendar_date': 'visit_date'})
date_info['visit_date'] = pd.to_datetime(date_info['visit_date'])
# date_info['dow'] = lbl.fit_transform(date_info['day_of_week'])
date_info['visit_date'] = date_info['visit_date'].dt.date

In [72]:
train = pd.merge(train, date_info[['visit_date', 'holiday_flg']], how='left', on=['visit_date']) 
test = pd.merge(test, date_info[['visit_date', 'holiday_flg']], how='left', on=['visit_date']) 

In [73]:
train.shape

(309585, 106)

# Use id as a feature

In [74]:
lbl = preprocessing.LabelEncoder()
train['le_air_store_id'] = lbl.fit_transform(train['air_store_id'])
test['le_air_store_id'] = lbl.transform(test['air_store_id'])

# col = [c for c in train if c not in ['id', 'air_store_id', 'visit_date','visitors']]
# train = train.fillna(-1)
# test = test.fillna(-1)

In [75]:
col = [c for c in train if c not in ['id','visitors']]

In [76]:
target = train['visitors']

In [77]:
# !mkdir stored_files

In [78]:
# train[col].to_pickle('./stored_files/24_12_train.pkl')

In [79]:
# test[col].to_pickle('./stored_files/24_12_test.pkl')

In [80]:
# target.to_pickle('./stored_files/24_12_target.pkl')

### ref

https://machinelearningmastery.com/time-series-data-stationary-python/